загруаем необходимые модули для работы

In [5]:
import pandas as pd
import numpy as np


открываем файлы тестовый и трейновых данных

In [3]:
test_df = pd.read_csv('TestDataFlight.csv')
train_df = pd.read_csv('TrainDataFlight.csv')

In [ ]:
train_df.head()

,Species,Length1,Length2,Length3,Height,Width
0,Hirundichthys,9.3,9.8,10.8,1.7388,1.0476
1,Fodiater,34.8,37.3,39.8,6.2884,4.0198
2,Parexocoetus,19.0,20.7,23.2,9.3960,3.4104
3,Cheilopogon,34.6,37.0,39.3,10.5717,6.3666
4,Exocoetus,21.1,22.5,25.0,6.4000,3.8000


In [11]:
X_train = train_df.drop(['Width'], axis = 1)
y_train = train_df['Width']
X_test = test_df.drop(['Width'], axis = 1)
y_test = test_df['Width']

мы видим Species - категориальные переменные, а Widht - масса нашего таргета , закодируем 

In [15]:
import category_encoders as ce
x_train_be = ce.BinaryEncoder(cols=['Species'])
x_test_be = ce.BinaryEncoder(cols=['Species'])
X_train = x_train_be.fit_transform(X_train)
X_test = x_test_be.fit_transform(X_test)

In [16]:
X_train 

,Species_0,Species_1,Species_2,Length1,Length2,Length3,Height,Weight
0,0,0,1,35.0,38.5,44.1,18.0369,920.0
1,0,1,0,10.0,10.5,11.6,1.9720,7.5
2,0,0,1,23.9,26.5,31.1,12.3778,340.0
3,0,1,1,24.0,26.0,29.2,8.8768,290.0
4,1,0,0,18.4,20.0,22.4,8.8928,150.0
...,...,...,...,...,...,...,...,...
114,1,0,0,19.0,20.7,23.2,8.5376,140.0
115,0,0,1,31.0,33.5,38.7,14.4738,650.0
116,1,1,0,20.0,22.0,23.5,6.1100,120.0
117,1,1,0,27.8,30.0,31.6,7.6156,320.0


In [18]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split    
from sklearn.linear_model import LinearRegression

In [ ]:
X_train,y_train,X_test,y_test = train_test_split(X_train,)